In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
import numpy as np

# use folder as train data
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

train_data = ImageFolder(root='images/impact_dataset', transform=ToTensor())
train_data_loader = DataLoader(train_data, batch_size=1, shuffle=True)





In [3]:
for X, y in train_data_loader:
    print('Shape of X:', X.shape)
    print('Shape of y:', y.shape, y.dtype)
    break
    

Shape of X: torch.Size([1, 3, 500, 500])
Shape of y: torch.Size([1]) torch.int64


In [4]:
# Utilisation de la GPU si disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [5]:

# Définition du modèle
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten() # convert 2D image to 1D vector
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 500 * 500, 512),  # Corrected input size
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 50),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=750000, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=50, bias=True)
    (5): ReLU()
  )
)


In [6]:
# Définition de la fonction de coût et de l'optimiseur
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


In [7]:

# Définition de la fonction d'entraînement
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        
        # Calcul de la prédiction et de la fonction de coût
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Rétropropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [8]:

# Définition de la fonction de test
def test(dataloader, model):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [9]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_data_loader, model, loss_fn, optimizer)
    test(train_data_loader, model)
print("Done!")


Epoch 1
-------------------------------
loss: 3.937113  [    0/ 3975]
loss: 3.912023  [  100/ 3975]
loss: 3.912023  [  200/ 3975]
loss: 3.912023  [  300/ 3975]
loss: 3.912023  [  400/ 3975]
loss: 3.912023  [  500/ 3975]
loss: 3.912023  [  600/ 3975]
loss: 3.912023  [  700/ 3975]
loss: 3.912023  [  800/ 3975]
loss: 3.912023  [  900/ 3975]
loss: 3.912023  [ 1000/ 3975]
loss: 3.912023  [ 1100/ 3975]
loss: 3.912023  [ 1200/ 3975]
loss: 3.912023  [ 1300/ 3975]
loss: 3.912023  [ 1400/ 3975]
loss: 3.912023  [ 1500/ 3975]
loss: 3.912023  [ 1600/ 3975]
loss: 3.912023  [ 1700/ 3975]
loss: 3.912023  [ 1800/ 3975]
loss: 3.912023  [ 1900/ 3975]
loss: 3.912023  [ 2000/ 3975]
loss: 3.912023  [ 2100/ 3975]
loss: 3.912023  [ 2200/ 3975]
loss: 3.912023  [ 2300/ 3975]
loss: 3.912023  [ 2400/ 3975]
loss: 3.912023  [ 2500/ 3975]
loss: 3.912023  [ 2600/ 3975]
loss: 3.912023  [ 2700/ 3975]
loss: 3.912023  [ 2800/ 3975]
loss: 3.912023  [ 2900/ 3975]
loss: 3.912023  [ 3000/ 3975]
loss: 3.912023  [ 3100/ 3975]


In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")


Saved PyTorch Model State to model.pth


In [11]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))



C:\Users\Paul\AppData\Local\Temp\ipykernel_27428\145263946.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth"))


<All keys matched successfully>